# Agenda: Day 2

1. Recap and Q&A
2. Dtypes and `NaN`
3. Data frames (2D data)
    - Creating data frames
    - Retrieving rows
    - Retrieving columns
    - Naming the index and the columns
4. Adding and removing data
5. Useful methods and attributes